In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns
import pickle 

In [23]:
data=pd.read_csv('heart.csv')
df_data=pd.DataFrame(data)
df_data



,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
df_data['Sex']=df_data['Sex'].map({'M':1,'F':0})
df_data['ChestPainType']=df_data['ChestPainType'].map({'ATA':0,'NAP':1,'ASY':2,'TA':3})
df_data['RestingECG']=df_data['RestingECG'].map({'Normal':0,'ST':1,'LVH':2})
df_data['ExerciseAngina']=df_data['ExerciseAngina'].map({'N':0,'Y':1})
df_data['ST_Slope']=df_data['ST_Slope'].map({'Up':0,'Flat':1,'Down':2})



In [ ]:
df_data.corr()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
Age,1.000000,0.055750,0.214164,0.254399,-0.095282,0.198039,0.213152,-0.382045,0.215793,0.258612,0.268264,0.282039
Sex,0.055750,1.000000,0.179766,0.005133,-0.200092,0.120076,-0.018343,-0.189186,0.190664,0.105734,0.150693,0.305445
ChestPainType,0.214164,0.179766,1.000000,0.075332,-0.130997,0.155111,0.112067,-0.273076,0.324049,0.300846,0.352364,0.459017
RestingBP,0.254399,0.005133,0.075332,1.000000,0.100893,0.070193,0.097661,-0.112135,0.155101,0.164803,0.075162,0.107589
Cholesterol,-0.095282,-0.200092,-0.130997,0.100893,1.000000,-0.260974,0.112095,0.235792,-0.034166,0.050148,-0.111471,-0.232741
FastingBS,0.198039,0.120076,0.155111,0.070193,-0.260974,1.000000,0.050707,-0.131438,0.060451,0.052698,0.175774,0.267291
RestingECG,0.213152,-0.018343,0.112067,0.097661,0.112095,0.050707,1.000000,0.048552,0.036119,0.114428,0.078807,0.061011
MaxHR,-0.382045,-0.189186,-0.273076,-0.112135,0.235792,-0.131438,0.048552,1.000000,-0.370425,-0.160691,-0.343419,-0.400421
ExerciseAngina,0.215793,0.190664,0.324049,0.155101,-0.034166,0.060451,0.036119,-0.370425,1.000000,0.408752,0.428706,0.494282
Oldpeak,0.258612,0.105734,0.300846,0.164803,0.050148,0.052698,0.114428,-0.160691,0.408752,1.000000,0.501921,0.403951


In [ ]:
df_data.head()

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(df_data.corr(),annot=True,linewidths=0.25)
plt.show()

In [100]:
cols_to_add=['ST_Slope','ExerciseAngina','ChestPainType']
x=df_data[cols_to_add]
y=df_data['HeartDisease']
x

,ST_Slope,ExerciseAngina,ChestPainType
0,0,0,0
1,1,0,1
2,0,0,0
3,1,1,2
4,0,0,1
...,...,...,...
913,1,0,3
914,1,0,2
915,1,1,2
916,1,0,0


In [101]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=10)


In [102]:
rand=RandomForestClassifier()
param_grid={'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]}
model=RandomizedSearchCV(rand,param_distributions=param_grid,n_iter=15,cv=5,scoring='accuracy')
model.fit(x_train,y_train)

,estimator,RandomForestClassifier()
,param_distributions,"{'bootstrap': [True, False], 'max_depth': [None, 10, ...], 'min_samples_leaf': [1, 2], 'min_samples_split': [2, 5], ...}"
,n_iter,15
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [104]:
#model=RandomForestClassifier()
#model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy_score(y_pred,y_test)


0.8434782608695652

In [106]:
with open('heartt.pkl','wb')as file:
    pickle.dump(model,file)